# 1404. Number of Steps to Reduce a Number in Binary Representation to One

Notes:
- Dave's second recursion :)
- For a binary number $b_{n-1} b_{n-2} \cdots b_1 b_0$, where $b_i$ is the $i^{th}$ bit from the right, the integer value $I$ can be computed as: $I = \sum_{i=0}^{n-1} b_i \times 2^i$ .

In [1]:
class Solution:
    def numSteps(self, s: str) -> int:
        """
        Given the binary representation of an integer as a string s, return the
        number of steps to reduce it to 1 under the following rules:

        - If the current number is even, you have to divide it by 2.
        - If the current number is odd, you have to add 1 to it.
        - It is guaranteed that you can always reach one for all test cases.
        """

        # Define a recursive function to count the number of steps.
        def process_binary(s: str, steps: int) -> int:
            # Base case if s = "1" no further steps are needed.
            if s == "1":
                return steps
            # If the integer is even (ie the string ends in a "0").
            if s[-1] == "0":
                # Divide by 2 (ie remove the last digit) and increment the step count.
                return process_binary(s[:-1], steps + 1)
            # Else, the integer is odd.
            else:
                # Binary -> integer conversion.
                num = int(s, 2)
                # Add 1 to the integer.
                num += 1
                # Integer -> binary conversion.
                new_s = bin(num)[2:]  # Note "[2:]" is to remove the "0b" prefix.
                return process_binary(new_s, steps + 1)

        return process_binary(s, 0)

In [2]:
# Run sanity checks.
s_list = [
    "1101",  # 6
    "10",  # 1
    "1",  # 0
    "11000",  # 6
]
for s in s_list:
    solution = Solution()
    print(solution.numSteps(s))

6
1
0
6


# 1442. Count Triplets That Can Form Two Arrays of Equal XOR

Notes:
- The XOR operation (denoted by `^` in Python and $\oplus$ in math) between two bits results in 1 if the bits are different, 0 if they are the same, and has several important properties.
  - Identity Property: $x \oplus 0 = x$, ie XORing a number with 0 leaves it unchanged.
  - Self-Inverse Property: $x \oplus x = 0$, ie XORing a number with itself results in 0.
  - Commutative Property: $x \oplus y = y \oplus x$, ie the order of XOR operations does not matter.
  - Associative Property: $(x \oplus y) \oplus z = x \oplus (y \oplus z)$, ie the grouping of XOR operations does not matter.
- Given an array of integers `arr`, the task is to find number of triplets `(i, j, k)` such that:
    - $0 \leq i < j \leq k < |\text{arr}|$
    -  The XOR of the elements from index `i` to `j-1` is equal to the XOR of the elements from index `j` to `k`.
    - $a = \text{arr}[i] \oplus \text{arr}[i + 1] \oplus \ldots \oplus \text{arr}[j - 1]$
    - $b = \text{arr}[j] \oplus \text{arr}[j + 1] \oplus \ldots \oplus \text{arr}[k]$
- The goal is to count the number of triplets where $a = b$, consequently $a = b \implies a \oplus b = 0$ $\therefore$ by the definitions of $a$ and $b$ and the properties of XOR operations:
$$(\text{arr}[i] \oplus \text{arr}[i + 1] \oplus \ldots \oplus \text{arr}[j - 1]) \oplus (\text{arr}[j] \oplus \text{arr}[j + 1] \oplus \ldots \oplus \text{arr}[k]) = \text{arr}[i] \oplus \text{arr}[i + 1] \oplus \ldots \oplus \text{arr}[k] = 0 \; .$$
- Let $\text{prefix\_xor}[l] = \text{arr}[0] \oplus \text{arr}[1] \oplus \cdots \oplus \text{arr}[l]$ denote the cumulative XOR from the beginning of the array up to the element at index `l`. Now consider the XOR of the subarray from `i` to `k`, $\text{arr}[i] \oplus \text{arr}[i+1] \oplus \cdots \oplus \text{arr}[k]$,  by the properties of XOR this can be written in terms of $\text{prefix\_xor}[l]$:
$$\text{arr}[i] \oplus \text{arr}[i+1] \oplus \cdots \oplus \text{arr}[k] = \text{prefix\_xor}[k] \oplus \text{prefix\_xor}[i-1] \implies \text{prefix\_xor}[k] \oplus \text{prefix\_xor}[i-1] = 0 \implies \text{prefix\_xor}[k] = \text{prefix\_xor}[i-1]$$
- With this in hand for each index `k`, the objective is determine how many indices `i` exist such that, $\text{prefix\_xor}[k] = \text{prefix\_xor}[i-1]$


In [3]:
class Solution:
    def countTriplets(self, arr: list[int]) -> int:
        """
        Given an array of integers arr. We want to select three indices i, j
        and k where (0 <= i < j <= k < arr.length). Let's define a and b as
        follows:
        - a = arr[i] ^ arr[i + 1] ^ ... ^ arr[j - 1]
        - b = arr[j] ^ arr[j + 1] ^ ... ^ arr[k]
        Note that ^ denotes the bitwise-xor operation. Return the number of
        triplets (i, j and k) Where a == b.
        """
        # Initialize the cumulative XOR up to the current element.
        prefix_xor = 0
        # Initialize the number of triplets.
        n_triplets = 0
        # Initialize a dictionary to keep track of how many times each prefix_xor value has been seen.
        prefix_counts = {0: 1}
        # Initialize a dictionary to keep track of the sum of indices where each prefix_xor value has been seen.
        prefix_sums = {
            0: -1
        }  # -1 is to account for the fact that the first element is at index 0.
        # Iterate through each element in the array.
        for k in range(len(arr)):
            # Update prefix_xor with the current element.
            prefix_xor ^= arr[k]
            # If the current prefix_xor has been seen before in prefix_counts.
            if prefix_xor in prefix_counts:
                # Calculate the number of valid triplets using the formula:
                # prefix_counts[prefix_xor] * k: Total number of possible subarrays ending at k.
                # prefix_sums[prefix_xor]: Sum of all previous indices where this prefix_xor was seen.
                # prefix_counts[prefix_xor]: Adjusting for the count of prefix_xor itself.
                n_triplets += (
                    prefix_counts[prefix_xor] * k
                    - prefix_sums[prefix_xor]
                    - prefix_counts[prefix_xor]
                )
                # Increment the count of current prefix_xor in prefix_counts.
                prefix_counts[prefix_xor] += 1
                # Update the sum of indices for the current prefix_xor in prefix_sums by adding the current index k.
                prefix_sums[prefix_xor] += k
            # Else, if the current prefix_xor has not been seen before.
            else:
                # Initialize the count of the new prefix_xor in prefix_counts.
                prefix_counts[prefix_xor] = 1
                # Initialize the sum of indices for the current prefix_xor to the current index k in prefix_sums.
                prefix_sums[prefix_xor] = k
        return n_triplets

In [4]:
# Run sanity checks.
arr_list = [
    [2, 3, 1, 6, 7],  # 4
    [1, 1, 1, 1, 1],  # 10
]
for arr in arr_list:
    solution = Solution()
    print(solution.countTriplets(arr))

4
10


# 260. Single Number III

In [5]:
class Solution:
    def singleNumber(self, nums: list[int]) -> list[int]:
        """
        Given an integer array nums, in which exactly two elements appear only
        once and all the other elements appear exactly twice. Find the two
        elements that appear only once. You can return the answer in any order.

        You must write an algorithm that runs in linear runtime complexity and
        uses only constant extra space.
        """
        # Initialize a set of unique numbers.
        unique_nums = set()
        # Iterate through each number in nums.
        for num in nums:
            # If the number has not been seen before.
            if num not in unique_nums:
                # Add it to the set.
                unique_nums.add(num)
            # Else, the number has been seen before.
            else:
                # Remove it from the set.
                unique_nums.remove(num)
        return list(unique_nums)

In [6]:
# Run sanity checks.
nums_list = [
    [1, 2, 1, 3, 2, 5],  # [3, 5] or [5, 3]
    [-1, 0],  # [-1, 0] or [0, -1]
    [0, 1],  # [0, 1] or [1, 0]
]
for nums in nums_list:
    solution = Solution()
    print(solution.singleNumber(nums))

[3, 5]
[0, -1]
[0, 1]


# 3110. Score of a String

In [1]:
class Solution:
    def scoreOfString(self, s: str) -> int:
        """
        You are given a string s. The score of a string is defined as the sum
        of the absolute difference between the ASCII values of adjacent
        characters.

        Return the score of s.
        """
        # Find the sum of the absolute differences between the ASCII values by
        # using zip to iterate through pairs of adjacent characters.
        return sum(abs(ord(a) - ord(b)) for a, b in zip(s, s[1:]))

In [2]:
# Run sanity checks.
s_list = [
    "hello",  # 13
    "zaz",  # 50
]
for s in s_list:
    solution = Solution()
    print(solution.scoreOfString(s))

13
50
